In [3]:
import numpy as np
import pandas as pd

from itertools import combinations, combinations_with_replacement
from tqdm import tqdm

In [9]:
animals_df = pd.DataFrame()
animals_df['Animal'] = list(animals.keys())
animals_df['Leaving 1'] = [animals[k][0] for k in animals.keys()]
animals_df['Leaving 2'] = [animals[k][1] for k in animals.keys()]
animals_df['Type'] = [animals[k][2] for k in animals.keys()]
animals_df.to_csv('animals.csv', index=False)
animals_df.head()

,Animal,Leaving 1,Leaving 2,Type
0,Alligator,Claw,Fang,TW
1,Apkallu,Fleece,Egg,C
2,Apkallu of Paradise,Egg,Fleece,T
3,Aurochs,Milk,Horn,C
4,Beachcomb,Carapace,Claw,TW


In [8]:
animals = {
    'Alligator' : ['Claw', 'Fang', 'TW'],
    'Apkallu' : ['Fleece', 'Egg', 'C'],
    'Apkallu of Paradise' : ['Egg', 'Fleece', 'T'],
    'Aurochs' : ['Milk', 'Horn', 'C'],
    'Beachcomb' : ['Carapace', 'Claw', 'TW'],
    'Black Chocobo' : ['Feather', 'Fur', 'W'],
    'Blue Back' : ['Egg', 'Feather', 'C'],
    'Chocobo' : ['Fur', 'Feather', 'C'],
    'Coblyn' : ['Fang', 'Carapace', 'C'],
    'Dodo' : ['Egg', 'Feather', 'C'],
    'Dodo of Paradise' : ['Feather', 'Egg', 'T'],
    'Glyptodon' : ['Claw', 'Carapace', 'T'],
    'Glyptodon Pup' : ['Carapace', 'Claw', 'C'],
    'Goobue' : ['Fang', 'Claw', 'TW'],
    'Gold Back' : ['Feather', 'Egg', 'W'],
    'Grand Buffalo' : ['Horn', 'Milk', 'W'],
    'Ground Squirrel' : ['Claw', 'Fur', 'C'],
    'Island Billy' : ['Horn', 'Fleece', 'T'],
    'Island Doe' : ['Fur', 'Milk', 'C'],
    'Island Nanny' : ['Milk', 'Horn', 'C'],
    'Island Stag' : ['Fur', 'Horn', 'T'],
    'Lemur' : ['Fur', 'Claw', 'T'],
    'Lost Lamb' : ['Fleece', 'Milk', 'C'],
    'Opo-Opo' : ['Claw', 'Fur', 'C'],
    'Ornery Karakul' : ['Milk', 'Fleece', 'W'],
    'Paissa' : ['Claw', 'Fleece', 'TW'],
    'Star Marmot' : ['Fur', 'Claw', 'T'],
    'Twinklefleece' : ['Fleece', 'Fur', 'TW'],
    'Yellow Coblyn' : ['Carapace', 'Fang', 'W']
}

rares = ['Alligator', 'Apkallu of Paradise', 'Beachcomb', 'Black Chocobo', 'Dodo of Paradise',
         'Glyptodon', 'Goobue', 'Gold Back', 'Grand Buffalo', 'Island Billy', 'Island Stag',
         'Lemur', 'Ornery Karakul', 'Paissa', 'Star Marmot', 'Twinklefleece', 'Yellow Coblyn']
nonrares = ['Apkallu', 'Aurochs', 'Blue Back', 'Chocobo', 'Coblyn', 'Dodo', 'Glyptodon Pup',
            'Ground Squirrel', 'Island Doe', 'Island Nanny', 'Lost Lamb', 'Opo-Opo']

extra_fangs = ['Coblyn']

all_pastures = list(combinations(rares + nonrares + extra_fangs, 20))

temp = rares.copy()
temp.remove('Paissa')

all_rare_pastures = [temp + list(x) for x in list(combinations_with_replacement(nonrares, 4))]

def get_leavings(comb):
    leavings = [animals[a] for a in comb]
    guaranteed, bonus = zip(*leavings)
    
    return guaranteed, bonus

def evaluate_pasture(comb, thresholds, min_each=2, chance=0.8):
    guaranteed, bonus = get_leavings(comb)
    
    g_labels, g_num = np.unique(guaranteed, return_counts=True)
    
    # Ensure we're getting 2 of each material guaranteed at minimum
    #   as a way to combat bad RNG
    
    if len(g_labels) < 9:
        return {}, False
    if min(g_num) < min_each:
        return {}, False
    
    b_labels, b_num = np.unique(bonus, return_counts=True)
    b_num = np.multiply(b_num, chance)
    
    # Get total
    total = dict()
    
    for i, _ in enumerate(g_labels):
        total[g_labels[i]] = g_num[i]
        
    for i, _ in enumerate(b_labels):
        total[b_labels[i]] += b_num[i]
    
    # If any of our totals are less than the threshold for any leaving, break
    for key in total.keys():
        if total[key] < thresholds[key]:
            return {}, False
        
    return total, True

def print_results(a_list, totals):
    print('--Pasture--')
    for a in sorted(a_list):
        print(f'| {a}')
    print('--Avg Leavings per Day')
    for t in totals.keys():
        print(f'| {t}-{round(totals[t],1)}')

len(all_pastures), len(all_rare_pastures)

(30045015, 1365)

# Manual Checks

In [10]:
a_list = ['Alligator', 'Apkallu of Paradise', 'Beachcomb', 'Coblyn', 'Dodo of Paradise', 'Glyptodon Pup', 'Gold Back',
          'Goobue', 'Grand Buffalo', 'Island Billy', 'Island Doe', 'Lemur', 'Lost Lamb', 'Ornery Karakul', 'Star Marmot',
          'Twinklefleece', 'Yellow Coblyn', 'Aurochs', 'Blue Back', 'Ground Squirrel']

dummy = {
    'Carapace': 1,
    'Claw': 1,
    'Egg': 1,
    'Fang': 1,
    'Feather': 1,
    'Fleece': 1,
    'Fur': 1,
    'Horn': 1,
    'Milk': 1,
}

t, _ = evaluate_pasture(a_list, dummy, min_each=1)
print_results(a_list, {k: v for k, v in reversed(sorted(t.items(), key=lambda item: item[1]))})

--Pasture--
| Alligator
| Apkallu of Paradise
| Aurochs
| Beachcomb
| Blue Back
| Coblyn
| Dodo of Paradise
| Glyptodon Pup
| Gold Back
| Goobue
| Grand Buffalo
| Ground Squirrel
| Island Billy
| Island Doe
| Lemur
| Lost Lamb
| Ornery Karakul
| Star Marmot
| Twinklefleece
| Yellow Coblyn
--Avg Leavings per Day
| Claw-6.0
| Fur-4.6
| Milk-4.4
| Fleece-4.4
| Carapace-3.8
| Fang-3.6
| Egg-3.6
| Horn-2.8
| Feather-2.8


# Any Combination of Rare/Nonrare (No Repeats)

In [20]:
# Make all pastures, then reduce it to only pastures that meet requirements
opt_thresholds = {
    'Carapace': 4,
    'Claw': 4,
    'Egg': 2.5,
    'Fang': 4,
    'Feather': 2.5,
    'Fleece': 3,
    'Fur': 4,
    'Horn': 2.5,
    'Milk': 3,
}

# Calculations
meets_goal = [False] * len(all_pastures)
best_setups_all = []

with tqdm(total=len(all_pastures)) as pbar:
    for i, x in enumerate(all_pastures):
        total, meets_goal[i] = evaluate_pasture(x, opt_thresholds)

        if meets_goal[i]:
            best_setups_all.append((all_pastures[i], total))
        pbar.update(1)
        
np.sum(meets_goal)

100%|███████████████████████████████████████████████████████████████████| 30045015/30045015 [16:16<00:00, 30781.44it/s]


1792

In [21]:
prior = ['Claw', 'Fur', 'Fang', 'Carapace']
non_prior = ['Feather', 'Horn', 'Egg']

scores = [0] * len(best_setups_all)

super_best = []

for i, (animals_list, totals) in enumerate(best_setups_all):
    totals = {k: v for k, v in reversed(sorted(totals.items(), key=lambda item: item[1]))}
    
    keys, vals = list(totals.keys()), list(totals.values())
    if all(item in keys[-3:] for item in non_prior) and all(item in keys[:4] for item in prior) \
    and 'Fang' in keys[:2] and all(totals[item] < 3 for item in non_prior):
#         print(keys, vals)
        super_best.append([animals_list, totals])

len(super_best)

16

In [31]:
# Get intersections
inters = [a for a, _ in super_best]
print(len(inters))
    
inter = sorted(list(set.intersection(*map(set,inters[-8:]))))

inter
# inter = ['Coblyn',
#  'Grand Buffalo',
#  'Glyptodon Pup',
#  'Beachcomb',
#  'Yellow Coblyn',
#  'Dodo of Paradise',
#  'Island Billy',
#  'Twinklefleece',
#  'Ornery Karakul',
#  'Gold Back',
#  'Goobue',
#  'Star Marmot',
#  'Alligator',
#  'Lemur',
#  'Island Doe',
#  'Lost Lamb',
#  'Apkallu of Paradise']

16


['Alligator',
 'Apkallu of Paradise',
 'Beachcomb',
 'Black Chocobo',
 'Coblyn',
 'Glyptodon Pup',
 'Gold Back',
 'Goobue',
 'Grand Buffalo',
 'Island Billy',
 'Lemur',
 'Lost Lamb',
 'Ornery Karakul',
 'Star Marmot',
 'Twinklefleece',
 'Yellow Coblyn']

In [34]:
for i, (a_list, totals) in enumerate(super_best[-8:]):
    temp = list(a_list).copy()
    for a in inter:
        temp.remove(a)
    print_results(temp, totals)
#     print(sorted(temp), list(totals.values()))
    print('')

--Pasture--
| Aurochs
| Blue Back
| Coblyn
| Ground Squirrel
--Avg Leavings per Day
| Claw-5.5
| Fang-4.4
| Carapace-4.4
| Fur-4.1
| Fleece-4.1
| Milk-3.4
| Horn-2.7
| Feather-2.7
| Egg-2.7

--Pasture--
| Aurochs
| Blue Back
| Coblyn
| Opo-Opo
--Avg Leavings per Day
| Claw-5.5
| Fang-4.4
| Carapace-4.4
| Fur-4.1
| Fleece-4.1
| Milk-3.4
| Horn-2.7
| Feather-2.7
| Egg-2.7

--Pasture--
| Aurochs
| Coblyn
| Dodo
| Ground Squirrel
--Avg Leavings per Day
| Claw-5.5
| Fang-4.4
| Carapace-4.4
| Fur-4.1
| Fleece-4.1
| Milk-3.4
| Horn-2.7
| Feather-2.7
| Egg-2.7

--Pasture--
| Aurochs
| Coblyn
| Dodo
| Opo-Opo
--Avg Leavings per Day
| Claw-5.5
| Fang-4.4
| Carapace-4.4
| Fur-4.1
| Fleece-4.1
| Milk-3.4
| Horn-2.7
| Feather-2.7
| Egg-2.7

--Pasture--
| Blue Back
| Coblyn
| Ground Squirrel
| Island Nanny
--Avg Leavings per Day
| Claw-5.5
| Fang-4.4
| Carapace-4.4
| Fur-4.1
| Fleece-4.1
| Milk-3.4
| Horn-2.7
| Feather-2.7
| Egg-2.7

--Pasture--
| Blue Back
| Coblyn
| Island Nanny
| Opo-Opo
--Avg Le

# All Rare Spawns in Pasture (Repeat Non-rares allowed)

In [107]:
# Make all pastures, then reduce it to only pastures that meet requirements
rare_thresholds = {
    'Carapace': 3,
    'Claw': 3,
    'Egg': 2,
    'Fang': 3,
    'Feather': 2,
    'Fleece': 2,
    'Fur': 3,
    'Horn': 2,
    'Milk': 2,
}

# Calculations
meets_goal = [False] * len(all_rare_pastures)
best_setups_rare = []

with tqdm(total=len(all_rare_pastures)) as pbar:
    for i, x in enumerate(all_rare_pastures):
        total, meets_goal[i] = evaluate_pasture(x, rare_thresholds, min_each=1)

        if meets_goal[i]:
            best_setups_rare.append((all_rare_pastures[i], total))
        pbar.update(1)
        
np.sum(meets_goal)

100%|███████████████████████████████████████████████████████████████████████████| 1365/1365 [00:00<00:00, 12864.60it/s]


244

In [109]:
prior = ['Fang', 'Claw', 'Fur', 'Carapace']
non_prior = ['Feather', 'Horn']

scores = [0] * len(best_setups_rare)

for i, (animals_list, totals) in enumerate(best_setups_rare):
    total_items = {k: v for k, v in reversed(sorted(totals.items(), key=lambda item: item[1]))}
    
    keys, vals = list(total_items.keys()), list(total_items.values())
    
    if all(item in keys[:5] for item in prior) and all(item in keys[-2:] for item in non_prior):
        print_results(animals_list[-4:], total_items)
        print('')

--Pasture--
| Apkallu
| Coblyn
| Island Doe
| Island Doe
--Avg Leavings per Day
| Fur-6.4
| Claw-4.8
| Fleece-4.1
| Fang-3.4
| Carapace-3.4
| Milk-3.1
| Egg-3.1
| Feather-3
| Horn-2.7

--Pasture--
| Apkallu
| Coblyn
| Island Doe
| Lost Lamb
--Avg Leavings per Day
| Fur-5.4
| Fleece-5.1
| Claw-4.8
| Fang-3.4
| Carapace-3.4
| Milk-3.1
| Egg-3.1
| Feather-3
| Horn-2.7

--Pasture--
| Apkallu
| Coblyn
| Lost Lamb
| Lost Lamb
--Avg Leavings per Day
| Fleece-6.1
| Claw-4.8
| Fur-4.4
| Fang-3.4
| Carapace-3.4
| Milk-3.1
| Egg-3.1
| Feather-3
| Horn-2.7

